In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob, os
import matplotlib.pyplot as plt
import time
import scipy.stats as stats
import matplotlib.dates as mdates
import re
import csv

In [2]:
# read in a user-created text file to point to local directories to avoid having to change this every time 
# we update code
lines=[]
with open('path_file.txt') as f:
    lines = f.readlines()
    
count = 0
for line in lines:
    count += 1
    index = line.find("=")
    #print(f'line {count}: {line}')
    #print(index)
    #print(line[0:index])
    line = line.rstrip()
    if line[0:index].find("argo")>=0:
        argo_path=line[index+1:]
    elif line[0:index].find("liar")>=0:
        liar_dir=line[index+1:]
    elif line[0:index].find("matlab")>=0:
        matlab_dir=line[index+1:]
        
# Set the paths
output_dir = 'output/'
data_dir = 'data/'

In [3]:
# get list of argo meta files
argolist = []
for file in os.listdir(argo_path):
    if file.endswith('meta.nc'):
        argolist.append(file)

print(len(argolist))

1839


In [4]:
cal_str_table = pd.DataFrame(columns=['wmo','o2_cal_comment', 'o2_cal_eq'])
# temp_list = ['3902471_meta.nc']
# loop through all, open meta files, read in calibration comments, store
for n, file in enumerate(argolist): #enumerate(temp_list): # 
    print(str(n) + ' ' + file)


    #also load meta file  for same float
    file_n = argo_path + file
    meta_n = xr.open_dataset(argo_path + file)
    # meta_n
    parameters_n = meta_n.PARAMETER.values
    # print(parameters_n)

    contains_doxy = any(parameter.decode('utf-8').strip() == 'DOXY' for parameter in parameters_n)
    if contains_doxy is False:
        continue
        print('no doxy')

    # not sure if i need this, but loading the Sprof file as well

    wmo_n = int((re.search(r'\d+', file)).group())

    
    sprof_n = xr.open_dataset(argo_path + str(wmo_n) + '_Sprof.nc')

    n_prof = sprof_n.dims['N_PROF']
    # gets order of sensors to extract calibration comments
    # some profiles might be missing sensor name (not sure why) so loop through looking
    o2_ind_all = np.full(n_prof, np.nan)
    # finds where o2 calibration comment is in each profile (at least for one float it changes from the first profile to the rest)
    for p in range(0, n_prof):
        cal_str = sprof_n.STATION_PARAMETERS.values.astype(str)[p]
        # print(cal_str)
        for i, param in enumerate(cal_str):
            if 'DOXY' in param:
                o2_ind_all[p] = i
                break
    
    # o2_ind = np.unique(o2_ind_all[~np.isnan(o2_ind_all)])
    # if len(o2_ind)>1:
    #     print('multiple o2 ind' + str(wmo_n))
    #     break
    o2_cal_full = []
    o2_eq_full = []

    for idx, o2_ind in enumerate(o2_ind_all):
        if ~np.isnan(o2_ind):
            o2_cal_full.append(sprof_n.SCIENTIFIC_CALIB_COMMENT.values[idx,-1,np.int32(o2_ind)])
            # pd.concat([o2_cal_full]+ sprof_n.SCIENTIFIC_CALIB_COMMENT.values[idx,-1,np.int32(o2_ind)], ignore_index=True)
            o2_eq_full.append(sprof_n.SCIENTIFIC_CALIB_EQUATION.values[idx,-1,np.int32(o2_ind)])

    data_comment_eq = pd.DataFrame({'o2_cal_full': o2_cal_full, 'o2_eq_full': o2_eq_full})

    # Drop duplicate rows to get unique pairs
    unique_pairs = data_comment_eq.drop_duplicates()
    # Create an empty list to store DataFrames
    data_frames = []
    # print(o2_cal_unique)
    # might have multiple unique comments, so save out each one:
    for i in unique_pairs.index:
        o2_cal_i = unique_pairs.o2_cal_full[i].decode("utf-8")
        o2_eq_i = unique_pairs.o2_eq_full[i].decode("utf-8")

        # print(o2_cal_i)
        # save with wmo only for now
        new_data_cal_info = pd.DataFrame({'wmo': [wmo_n],
                                        'o2_cal_comment': [o2_cal_i],
                                        'o2_cal_eq': [o2_eq_i]})
        # Append the new_data_cal_info DataFrame to the list
        data_frames.append(new_data_cal_info)

        # cal_str_table = cal_str_table.append(new_data_cal_info, ignore_index=True)
    # print(data_frames)
    # Concatenate all DataFrames in the list into a single DataFrame
    cal_str_table = pd.concat([cal_str_table] + data_frames, ignore_index=True)
    # print(cal_str_table)
    # if n==10:
    #     break
    # if wmo_n== 3902471:
    #     break
print(cal_str_table)

0 4901216_meta.nc
1 7901019_meta.nc
2 5904485_meta.nc
3 6903878_meta.nc
4 7901009_meta.nc
5 3902471_meta.nc
6 6900874_meta.nc
7 2900787_meta.nc
8 5906972_meta.nc
9 5905381_meta.nc
10 5906042_meta.nc
11 6902734_meta.nc
12 2903454_meta.nc
13 6902019_meta.nc
14 6902980_meta.nc
15 1901153_meta.nc
16 5901739_meta.nc
17 5906571_meta.nc
18 2903167_meta.nc
19 5906561_meta.nc
20 2902238_meta.nc
21 5905136_meta.nc
22 4900869_meta.nc
23 6903030_meta.nc
24 5901372_meta.nc
25 2900448_meta.nc
26 4900879_meta.nc
27 6903042_meta.nc
28 2900542_meta.nc
29 6902548_meta.nc
30 5902112_meta.nc
31 6901632_meta.nc
32 5901310_meta.nc
33 5906513_meta.nc
34 6901896_meta.nc
35 5906503_meta.nc
36 6901886_meta.nc
37 6903571_meta.nc
38 2900119_meta.nc
39 5906020_meta.nc
40 5905232_meta.nc
41 5906030_meta.nc
42 5903616_meta.nc
43 3902130_meta.nc
44 3902120_meta.nc
45 4902555_meta.nc
46 6901485_meta.nc
47 6902687_meta.nc
48 2902195_meta.nc
49 5906868_meta.nc
50 5905095_meta.nc
51 4902633_meta.nc
52 6903183_meta.nc
53 

In [5]:
cal_str_table

,wmo,o2_cal_comment,o2_cal_eq
0,4901216,DOXY_ADJUSTED corrected based on the WOA 2018 ...,PSAT = f(DOXY); PSAT_ADJUSTED = G*PSAT; DOXY_A...
1,4901216,Bad data; not adjustable ...,none ...
2,7901019,...,...
3,5904485,DOXY_ADJUSTED corrected using continuous in-ai...,PSAT = f(DOXY); PSAT_ADJUSTED = G*PSAT; DOXY_A...
4,5904485,none ...,none ...
...,...,...,...
2469,4900320,DOXY_ADJUSTED corrected based on the WOA 2018 ...,PSAT = f(DOXY); PSAT_ADJUSTED = G*PSAT; DOXY_A...
2470,4900320,none ...,none ...
2471,4900320,Bad data; not adjustable ...,none ...
2472,6901490,"Adjusted on CTD at deployment, optode simple c...",PPOX_ADJUSTED=OFFSET+(PPOX*SLOPE)*(1+DRIFT/100...


In [241]:
unique_cal_comments = np.unique(cal_str_table.o2_cal_comment)
wmo_comment_data_table = []
for comm in unique_cal_comments:
    n_wmo_for_comment = len(np.unique(cal_str_table.wmo[cal_str_table.o2_cal_comment==comm]))
    # print(n_wmo_for_comment)

    # Create a sublist for each row and append it to wmo_comment_data_table
    row = [comm, n_wmo_for_comment]
    wmo_comment_data_table.append(row)

    # if len(wmo_comment_data_table)==0:
    #     wmo_comment_data_table = [comm, n_wmo_for_comment]
    # else:
    #     wmo_comment_data_table.append([comm, n_wmo_for_comment])



In [242]:
wmo_comment_data_table

[['                                                                                                                                                                                                                                                                ',
  381],
 ['1-point multiplicative correction using WOD at 1008.49 dbar. The quoted error was computed via comparisons with monthly or annual climatology data, interpolated to float location, depth, and season, from WOA09.                                              ',
  1],
 ['1-point multiplicative correction using WOD at 1269.58 dbar. The quoted error was computed via comparisons with monthly or annual climatology data, interpolated to float location, depth, and season, from WOA09.                                              ',
  1],
 ['1-point multiplicative correction using WOD at 1792.895 dbar. The quoted error was computed via comparisons with monthly or annual climatology data, interpolated to float location, depth, and

In [235]:
df = pd.DataFrame(wmo_comment_data_table)
csv_file=  'wmo_comment_data_table.csv'

In [243]:
# Write the list to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(wmo_comment_data_table)

In [227]:
o2_offset_data_table = []
len(o2_offset_data_table)

0

In [208]:
o2_cal_full = []
o2_eq_full = []

for idx, o2_ind in enumerate(o2_ind_all):
    if ~np.isnan(o2_ind):
        o2_cal_full.append(sprof_n.SCIENTIFIC_CALIB_COMMENT.values[idx,-1,np.int32(o2_ind)])
        # pd.concat([o2_cal_full]+ sprof_n.SCIENTIFIC_CALIB_COMMENT.values[idx,-1,np.int32(o2_ind)], ignore_index=True)
        o2_eq_full.append(sprof_n.SCIENTIFIC_CALIB_EQUATION.values[idx,-1,np.int32(o2_ind)])

    # print(o2_cal_full)
data_comment_eq = pd.DataFrame({'o2_cal_full': o2_cal_full, 'o2_eq_full': o2_eq_full})
unique_pairs = data_comment_eq.drop_duplicates()

print(unique_pairs)


                                         o2_cal_full  \
0  b'DOXY_ADJUSTED is computed from an adjustment...   

                                          o2_eq_full  
0  b'PPOX_DOXY=f(DOXY), PPOX_DOXY_ADJUSTED=(SLOPE...  


In [202]:
len(o2_cal_full)

30

In [192]:
tt = sprof_n.SCIENTIFIC_CALIB_COMMENT.values[1,-1,np.int32(o2_ind)]

print(tt)

b'DOXY_ADJUSTED is computed from an adjustment of in water PSAT or PPOX float data at surface by comparison to woaPSAT climatology or woaPPOX{woaPSAT,floatTEMP,floatPSAL} at 1 atm, DOXY_ADJUSTED_ERROR is computed from a PPOX_ERROR of 10.0 mbar +1mb/year     '


In [144]:
p = 1
cal_str = sprof_n.STATION_PARAMETERS.values.astype(str)[p]
print(cal_str)

['PRES                                                            '
 'TEMP                                                            '
 'PSAL                                                            '
 'DOXY                                                            '
 'DOWN_IRRADIANCE380                                              '
 'DOWN_IRRADIANCE412                                              '
 'DOWN_IRRADIANCE490                                              '
 'DOWNWELLING_PAR                                                 '
 'CHLA                                                            '
 'BBP700                                                          '
 'CDOM                                                            '
 'PH_IN_SITU_TOTAL                                                ']


In [97]:
o2_cal_full = sprof_n.SCIENTIFIC_CALIB_COMMENT.values[:,-1,o2_ind]
o2_eq_full = sprof_n.SCIENTIFIC_CALIB_EQUATION.values[:,-1,o2_ind]

data_comment_eq = pd.DataFrame({'o2_cal_full': o2_cal_full, 'o2_eq_full': o2_eq_full})

# Drop duplicate rows to get unique pairs
unique_pairs = data_comment_eq.drop_duplicates()

print(unique_pairs.o2_cal_full[0])

b'DOXY_ADJUSTED corrected using continuous in-air measurements as in Johnson et al. (2015)                                                                                                                                                                        '


In [90]:
# o2_eq = sprof_n.SCIENTIFIC_CALIB_EQUATION.values[0,-1,o2_ind].decode("utf-8")
# 
o2_eq = np.unique(sprof_n.SCIENTIFIC_CALIB_EQUATION.values[:,-1,o2_ind])
print(o2_eq)
print(o2_cal)

[b'PSAT = f(DOXY); PSAT_ADJUSTED = G*PSAT; DOXY_ADJUSTED = f(PSAT_ADJUSTED)                                                                                                                                                                                        '
 b'none                                                                                                                                                                                                                                                            ']
[b'Bad data; not adjustable                                                                                                                                                                                                                                        '
 b'DOXY_ADJUSTED corrected using continuous in-air measurements as in Johnson et al. (2015)                                                                                                                             

In [61]:
# cal_str_table = pd.DataFrame(columns=['wmo','o2_cal_comment'])

#also load meta file  for same float
file_n = argo_path + file
meta_n = xr.open_dataset(argo_path + file)
# meta_n
parameters_n = meta_n.PARAMETER.values
# print(parameters_n)

contains_doxy = any(parameter.decode('utf-8').strip() == 'DOXY' for parameter in parameters_n)
if contains_doxy is False:
    # continue
    print('no doxy')

# not sure if i need this, but loading the Sprof file as well

wmo_n = int((re.search(r'\d+', file)).group())

sprof_n = xr.open_dataset(argo_path + str(wmo_n) + '_Sprof.nc')

# gets order of sensors to extract calibration comments
cal_str = sprof_n.STATION_PARAMETERS.values.astype(str)[0]
# print(cal_str)

for i, param in enumerate(cal_str):
    if 'DOXY' in param:
        o2_ind = i
        break
# print(i)
o2_cal_full = sprof_n.SCIENTIFIC_CALIB_COMMENT.values[:,-1,o2_ind]
o2_cal_unique = np.unique(o2_cal_full)

# print(o2_cal_unique)
# might have multiple unique comments, so save out each one:
for i in range(len(o2_cal_unique)):
    o2_cal_i = o2_cal_unique[i].decode("utf-8")
    print(o2_cal_i)
    # save with wmo only for now
    new_data_cal_info = pd.DataFrame({'wmo': [wmo_n],
                                      'o2_cal_comment': [o2_cal_i]})
    # Append the new_data_cal_info DataFrame to cal_str_table
    cal_str_table = cal_str_table.append(new_data_cal_info, ignore_index=True)


/var/folders/7r/4f_w_nb56llcwh96t_pjq_qh0000gn/T/ipykernel_86162/3566382346.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cal_str_table = cal_str_table.append(new_data_cal_info, ignore_index=True)


In [38]:
cal_str.

array(['PRES                                                            ',
       'TEMP                                                            ',
       'PSAL                                                            ',
       'DOXY                                                            '],
      dtype='<U64')

[b'                                                                                                                                                                                                                                                                ']
                                                                                                                                                                                                                                                                


In [19]:
parameters_n = meta_n.PARAMETER.values
print(parameters_n)

contains_doxy = any(parameter.decode('utf-8').strip() == 'DOXY' for parameter in parameters_n)
contains_doxy


[b'PRES                                                            '
 b'TEMP                                                            '
 b'PSAL                                                            '
 b'C1PHASE_DOXY                                                    '
 b'C2PHASE_DOXY                                                    '
 b'DOXY                                                            '
 b'TEMP_DOXY                                                       '
 b'PPOX_DOXY                                                       ']


True